In [16]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import ConversationChain, HypotheticalDocumentEmbedder, LLMChain, RetrievalQA
from domino_data.vectordb import DominoPineconeConfiguration
#from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone
from langchain.memory import ConversationSummaryMemory
from langchain import PromptTemplate
from langchain_community.chat_models import ChatMlflow
from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain_community.embeddings import CohereEmbeddings
from langchain_community.embeddings import MlflowEmbeddings
from langchain.retrievers import ContextualCompressionRetriever, CohereRagRetriever
from langchain_cohere import CohereRerank

import os
import pinecone
import sys
from mlflow.deployments import get_deploy_client

import warnings
warnings.filterwarnings('ignore')

client = get_deploy_client(os.environ['DOMINO_MLFLOW_DEPLOYMENTS'])

In [11]:
# initialize embedding
model = 'text-embedding-ada-002'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY') 
# The OpenAIEmbeddings class is instantiated with two parameters: 
# 'model' and 'openai_api_key'. 'model' is the name of the model to be used 
# and 'openai_api_key' is the key for accessing the OpenAI API.
embeddings = OpenAIEmbeddings(
    model=model,
    openai_api_key=OPENAI_API_KEY
)

In [15]:
embeddings = MlflowEmbeddings(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="embedding-ada-002ja2"
)

In [12]:
PINECONE_ENV=os.getenv('PINECONE_ENV')
COHERE_API_KEY=os.getenv('COHERE_API_KEY')

In [3]:
cohere_embeddings = CohereEmbeddings(cohere_api_key=COHERE_API_KEY)

In [13]:
# Domino Pinecone Data Source name 
datasource_name = "mrag-fin-docs-ja"

# Load Domino Pinecone Data Source Configuration 
conf = DominoPineconeConfiguration(datasource=datasource_name)

# api_key variable is a mandatory non-empty field used for 
# the native pinecone python client initialization 
# using Pinecone Data Source name 
api_key = os.environ.get("DOMINO_VECTOR_DB_METADATA", datasource_name)

# initialize pinecone
pinecone.init(
    api_key=api_key, 
    environment=PINECONE_ENV,
    openapi_config=conf # Domino Pinecone Data Source Configuration 
)

# Load Pinecone Index
index_name = "mrag-fin-docs"
index = pinecone.Index(index_name)
text_field = "text"
#vectorstore = Pinecone.from_existing_index(index_name, embeddings.embed_query)
# switch back to normal index for langchain
vectorstore = Pinecone(
    index, embeddings, text_field # Using embedded data from Domino AI Gateway Endpoint
)

In [14]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00361,
 'namespaces': {'': {'vector_count': 361}},
 'total_vector_count': 361}

In [17]:
# Create Cohere's reranker with the vector DB using Cohere's embeddings as the base retriever
cohere_rerank = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank, 
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 5})
)

compressed_docs = compression_retriever.get_relevant_documents("How many sales?")

TypeError: BaseCohere.rerank() takes 1 positional argument but 4 positional arguments (and 2 keyword-only arguments) were given

In [21]:
chatLLM = ChatMlflow(
        target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
        endpoint="chat-gpt4-ja", 
        temperature=0.0,
    )

In [22]:

prompt_template = """You are an AI assistant with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. 
When interacting, adhere to the following guidelines:
You are given the following extracted parts of a long document and a question. 
If you don't know the answer, just say "Hmm, I'm not sure." Don't try to make up an answer.
If the question is not related to corporate filings analysis, politely inform them that you are tuned to only answer questions pertaining to corporate filings analysis.
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction. pertaining to policy coverage.
Question: {question}
=========
{context}
=========
Answer in Markdown:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["question", "context"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [23]:


conversationChat = ConversationChain(
    llm=chatLLM,
    
    memory=ConversationSummaryMemory(llm=chatLLM),
    verbose=False
)

messages = [
    {"role": "assistant", "content": "How can I help you today?"}
]

In [24]:
qa_chain = RetrievalQA.from_chain_type(llm=chatLLM,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [36]:
user_question = input("Please ask your question:")
result = qa_chain(user_question)

Please ask your question: What was the Company’s total net sales increase during the first quarter of FY2024 


In [37]:
result["result"]

"The company's total net sales increased by 2% or $2.4 billion during the first quarter of 2024 compared to the same quarter in 2023. This increase was primarily driven by higher net sales of iPhone and Services, although it was partially offset by lower net sales of iPad and Wearables, Home and Accessories. The company also repurchased $20.5 billion of its common stock and paid dividends and dividend equivalents of $3.8 billion during the first quarter of 2024. \n\nHowever, it's important to note that the company's financial performance is subject to risks associated with changes in the value of the U.S. dollar relative to local currencies. Macroeconomic conditions, including inflation, changes in interest rates, and currency fluctuations, have directly and indirectly impacted, and could in the future materially impact, the company's results of operations and financial condition. \n\nIn terms of regional performance, net sales increased in Americas, Europe, Japan, and the rest of Asia

In [27]:
user_question = input("Please ask your financial question:")
compressed_docs = compression_retriever.get_relevant_documents(user_question)
# Print the relevant documents from using the embeddings and reranker
print(compressed_docs)

Please ask your financial question: Were there any product announcements in q1 of 2024?


NameError: name 'compression_retriever' is not defined

In [ ]:
# Setup HyDE

hyde_prompt_template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines:
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction. 
"Please answer the user's question below \n 
Question: {question}
Answer:
"""

hyde_prompt = PromptTemplate(input_variables=["question"], template=hyde_prompt_template)
hyde_llm_chain = LLMChain(llm=chatLLM, prompt=hyde_prompt)

hyde_embeddings = HypotheticalDocumentEmbedder(
    llm_chain=hyde_llm_chain, base_embeddings=embeddings
)

In [ ]:
# Get relevant docs through vector DB

SIMILARITY_THRESHOLD = 0.5

# Number of texts to match (may be less if no suitable match)
NUM_TEXT_MATCHES = 5

# Number of texts to return from reranking
NUM_RERANKING_MATCHES = 3

# Create prompt
template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines:If the question is not related to the context of corporate filings, politely respond with, 'Hi, I'm here to help analyze financial documents and related inquiries. Could you ask a question related to the company's corporate filings?'
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction. pertaining to policy coverage.
Here is some relevant context: {context}
"""

# Load the reranking model
colbert = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# Get relevant docs through vector DB
def get_relevant_docs(user_input, num_matches=NUM_TEXT_MATCHES, use_hyde=False):
   
    if use_hyde:
        embedded_query = hyde_embeddings.embed_query(user_input)
    else:
        embedded_query = embeddings.embed_query(user_input)
        
 
    relevant_docs = index.query(
        vector=embedded_query,
        top_k=num_matches,
        include_values=True,
        include_metadata=True
    )

    matches = relevant_docs["matches"]
    filtered_matches = [match for match in matches if match['score'] >= SIMILARITY_THRESHOLD]
    relevant_docs["matches"] = filtered_matches

    return relevant_docs

 
def build_system_prompt(user_input, rerank=False, use_hyde=False):
    print(user_input)
    try:
        relevant_docs = get_relevant_docs(user_input)
    except Exception as e:
        print(f"Failed to get relevant documents: {e}")
        return "", "Failed to get relevant documents"

    actual_num_matches = len(relevant_docs["matches"])
    if actual_num_matches == 0:
        print("No matches found in relevant documents.")
        return "", "No matches found in relevant documents"
    
    contexts = [relevant_docs["matches"][i]["metadata"]["text"] for i in range(actual_num_matches)]
    print("num_matches: ", actual_num_matches)
    if rerank and actual_num_matches >= NUM_RERANKING_MATCHES:
        try:
            docs = colbert.rerank(query=user_input, documents=contexts, k=NUM_RERANKING_MATCHES)
        except Exception as e:
            print(f"Failed to rerank documents: {e}")
            return "", "Failed to rerank documents"
        
        try:
            result_indices = [docs[i]["result_index"] for i in range(NUM_RERANKING_MATCHES)]
        except (IndexError, KeyError) as e:
                print(f"Invalid result indices: {e}")
                return "", "Invalid result indices"
        try:    
            contexts = [contexts[index] for index in result_indices]
        except IndexError as e:
            print(f"Indexing error: {e}")
            return "", "Indexing error"
    
    # Create prompt
    template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction.
Here is some relevant context: {context}"""
    
    prompt_template = PromptTemplate(
        input_variables=["context"],
        template=template
    )
    
    system_prompt = prompt_template.format(context=contexts)
    print(contexts)
    return system_prompt, contexts

# Query the Open AI Model
def queryAIModel(user_input, llm_name="openai", use_hyde=False):

    if not user_input:
        return "Please provide an input"
    
    system_prompt = build_system_prompt(user_input)
   # print(system_prompt)
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=user_input
        ),
    ]
    Print("hi")
    output = conversationChat.predict(input=messages)

    return output

In [ ]:
# Ask a question ; uncomment to test
user_question = input("Please provide your question here :")
result = queryAIModel(user_question)


In [ ]:
print(result)

In [10]:
index.describe_index_stats()


NameError: name 'index' is not defined

In [18]:
%run /mnt/code/services/data_ingestion.py


File /mnt/code/data/apple-10Q-2023Q4.pdf has been added.
